# Titanic Suvival Prediction
### by Logistic Regrssion using R Programming

## Prepare data and environment

In [ ]:
# install packages
install.packages("titanic")
library(titanic)
library(tidyverse)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
data("titanic_train")
glimpse(titanic_train)

Rows: 891
Columns: 12
$ PassengerId <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,…
$ Survived    <int> 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1…
$ Pclass      <int> 3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 1, 3, 3, 3, 2, 3, 2, 3, 3…
$ Name        <chr> "Braund, Mr. Owen Harris", "Cumings, Mrs. John Bradley (Fl…
$ Sex         <chr> "male", "female", "female", "female", "male", "male", "mal…
$ Age         <dbl> 22, 38, 26, 35, 35, NA, 54, 2, 27, 14, 4, 58, 20, 39, 14, …
$ SibSp       <int> 1, 1, 0, 1, 0, 0, 0, 3, 0, 1, 1, 0, 0, 1, 0, 0, 4, 0, 1, 0…
$ Parch       <int> 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 5, 0, 0, 1, 0, 0, 0…
$ Ticket      <chr> "A/5 21171", "PC 17599", "STON/O2. 3101282", "113803", "37…
$ Fare        <dbl> 7.2500, 71.2833, 7.9250, 53.1000, 8.0500, 8.4583, 51.8625,…
$ Cabin       <chr> "", "C85", "", "C123", "", "", "E46", "", "", "", "G6", "C…
$ Embarked    <chr> "S", "C", "S", "S", "S", "Q", "S", "S", "S", "C", "S", "S"…


In [ ]:
# Replace N/A with average age

titanic_train_clean <- na.omit(titanic_train)

meanAge <- round(mean(titanic_train_clean$Age))
titanic_train <- titanic_train %>%
  replace_na(list(Age = meanAge))

In [ ]:
# Convert the data type of the Survived column into factor

titanic_survived <- titanic_train %>% 
  mutate(Survived = factor(Survived,
                           levels = c(0, 1),
                           labels = c("Died", "Survived")))

In [ ]:
# Split data to 70% Train data and 30% Test data

n_row <- nrow(titanic_survived)

set.seed(75)
id <- sample(1:n_row, size = n_row*0.80)

train_data <- titanic_survived[id, ]
test_data  <- titanic_survived[-id, ]

In [ ]:
head(train_data)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,<int>,<fct>,<int>,<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<dbl>,<chr>,<chr>
704,704,Died,3,"Gallagher, Mr. Martin",male,25,0,0,36864,7.7417,,Q
168,168,Died,3,"Skoog, Mrs. William (Anna Bernhardina Karlsson)",female,45,1,4,347088,27.9000,,S
793,793,Died,3,"Sage, Miss. Stella Anna",female,30,8,2,CA. 2343,69.5500,,S
789,789,Survived,3,"Dean, Master. Bertram Vere",male,1,1,2,C.A. 2315,20.5750,,S
873,873,Died,1,"Carlsson, Mr. Frans Olof",male,33,0,0,695,5.0000,B51 B53 B55,S
759,759,Died,3,"Theobald, Mr. Thomas Leonard",male,34,0,0,363294,8.0500,,S


## Logistic Regression Procedure

In [ ]:
## Train Model

# Fit Model
logModel <- glm(Survived ~ Pclass + Sex + Age + SibSp + Parch,
                data = train_data, family = "binomial")
summary(logModel)


Call:
glm(formula = Survived ~ Pclass + Sex + Age + SibSp + Parch, 
    family = "binomial", data = train_data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1201  -0.6104  -0.4097   0.5947   2.4312  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  5.533427   0.553567   9.996  < 2e-16 ***
Pclass      -1.244809   0.135377  -9.195  < 2e-16 ***
Sexmale     -2.763267   0.225886 -12.233  < 2e-16 ***
Age         -0.043057   0.008889  -4.844 1.27e-06 ***
SibSp       -0.402414   0.125326  -3.211  0.00132 ** 
Parch       -0.055727   0.127859  -0.436  0.66295    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 947.98  on 711  degrees of freedom
Residual deviance: 624.87  on 706  degrees of freedom
AIC: 636.87

Number of Fisher Scoring iterations: 5


In [ ]:
## Train Model
# Probablity of survival of test data

train_data$Prob_survived <- predict(logModel, type = "response")
train_data$Pred_survived <- ifelse(train_data$Prob_survived >= 0.5, "Survived", "Died")

# Compare the reults of prediction with recorded data
train_data %>%
  select(PassengerId, Survived, Prob_survived, Pred_survived) %>%
  head()

,PassengerId,Survived,Prob_survived,Pred_survived
,<int>,<fct>,<dbl>,<chr>
704,704,Died,0.11499659,Died
168,168,Died,0.31780348,Died
793,793,Died,0.05606866,Died
789,789,Survived,0.17928622,Died
873,873,Died,0.52609442,Survived
759,759,Died,0.08104731,Died


In [ ]:
## Test Model
# Probablity of survival of test data

test_data$Prob_survived <- predict(logModel, newdata = test_data, type = "response")
test_data$Pred_survived <- ifelse(test_data$Prob_survived >= 0.5, "Survived", "Died")

# Compare the reults of prediction to recorded data
test_data %>%
  select(PassengerId, Survived, Prob_survived, Pred_survived) %>%
  head()

,PassengerId,Survived,Prob_survived,Pred_survived
,<int>,<fct>,<dbl>,<chr>
1,1,Died,0.08997563,Died
4,4,Survived,0.91522824,Survived
10,10,Survived,0.88479047,Survived
12,12,Survived,0.85708848,Survived
15,15,Died,0.76784882,Survived
16,16,Survived,0.66277331,Survived


# Model Evaluation

## Evaluate Train Data

In [ ]:
## Train confusion matrix
conMat_train <- table(train_data$Pred_survived, train_data$Survived,
                      dnn = c("Predict", "Actual"))

In [ ]:
# creat Train DataFrame

trainAccuracy <- (conMat_train[1, 1] + conMat_train[2, 2])/sum(conMat_train)
trainPrecision <- conMat_train[2, 2]/(conMat_train[2, 1] + conMat_train[2, 2])
trainRecall <- conMat_train[2, 2]/(conMat_train[1, 2] + conMat_train[2, 2])
trainF1score <- 2* ((trainRecall * trainPrecision)/(trainRecall + trainPrecision))

## Train Model Evaluation
Train <- data.frame(Model = "Train_Model",
                    Accuracy = trainAccuracy,
                    Precision = trainPrecision,
                    F1_Score = trainF1score)

cat("Train Confusion Metrix:\n")
print(conMat_train)
cat("\n")
cat("Train Model Evaluation:\n")
print(Train)

Train Confusion Metrix:
          Actual
Predict    Died Survived
  Died      376       76
  Survived   63      197

Train Model Evaluation:
        Model  Accuracy Precision F1_Score
1 Train_Model 0.8047753 0.7576923 0.739212


## Evaluate Test Data

In [ ]:
## Test confusion matrix
conMat_test <- table(test_data$Pred_survived, test_data$Survived,
                     dnn = c("Predict", "Actual"))

In [ ]:
# creat Test DataFrame

testAccuracy <- (conMat_test[1, 1] + conMat_test[2, 2])/sum(conMat_test)
testPrecision <- conMat_test[2, 2]/(conMat_test[2, 1] + conMat_test[2, 2])
testRecall <- conMat_test[2, 2]/(conMat_test[1, 2] + conMat_test[2, 2])
testF1score <- 2* ((testRecall * testPrecision)/(testRecall + testPrecision))

## Train Model Evaluation
Test <- data.frame(Model = "Test_Model",
                   Accuracy = testAccuracy,
                   Precision = testPrecision,
                   F1_Score = testF1score)

cat("Test Confusion Metrix:\n")
print(conMat_test)
cat("\n")
cat("Test Model Evaluation:\n")
print(Test)

Test Confusion Metrix:
          Actual
Predict    Died Survived
  Died       89       17
  Survived   21       52

Test Model Evaluation:
       Model  Accuracy Precision  F1_Score
1 Test_Model 0.7877095 0.7123288 0.7323944


# Summarisation

In [ ]:
TrainTest <- bind_rows(Train, Test)
tibble(TrainTest)

Model,Accuracy,Precision,F1_Score
<chr>,<dbl>,<dbl>,<dbl>
Train_Model,0.8047753,0.7576923,0.7392120
Test_Model,0.7877095,0.7123288,0.7323944
